In [ ]:
!bash drive/My\ Drive/datawithcred.sh

In [ ]:
from pathlib import Path
import tensorflow as tf
from src.data.make_dataset import GetDataset
from src.core.cloud import CloudTrainer

In [ ]:
tf.random.set_seed(123)

In [ ]:
dt = GetDataset(image_dims=(224, 224))

In [ ]:
train = dt.make_train_data(Path("chest_xray/train"))

In [ ]:
val = dt.make_train_data(Path("chest_xray/test"))

In [ ]:
trainer = CloudTrainer(experiment_name="VGGNet16", save_freq=30)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=./drive/My\ Drive/models --port=6006

In [ ]:
vgg = tf.keras.applications.VGG16(include_top=False, 
                                    weights=None, 
                                    input_shape=(dt.width, dt.height, 3), 
                                    pooling='max')
model = tf.keras.Sequential()
for layer in vgg.layers:
    layer.trainable = True
    model.add(layer)
# vgg = tf.keras.Model(vgg.input, vgg.layers[-1].output)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(4096, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(4096, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
# model = tf.keras.Sequential([
#     vgg,
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(4096, activation="relu"),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(4096, activation="relu"),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1, activation="sigmoid")
# ])

In [ ]:
model.summary()

In [ ]:
def get_model():
    vgg = tf.keras.applications.VGG16(include_top=False, 
                                    weights=None, 
                                    input_shape=(dt.width, dt.height, 3), 
                                    pooling='max')
    for layer in vgg.layers:
        layer.trainable = True
    vgg = tf.keras.Model(input=vgg.input, output=vgg.output)
    model = tf.keras.Sequential([
        vgg,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return model

In [ ]:
def get_model():
    vgg = tf.keras.applications.VGG16(include_top=False, 
                                    weights=None, 
                                    input_shape=(dt.width, dt.height, 3), 
                                    pooling='avg')
    model = tf.keras.Sequential()
    for layer in vgg.layers:
        layer.trainable = True
        model.add(layer)
    # vgg = tf.keras.Model(vgg.input, vgg.layers[-1].output)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(4096, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))
    return model

In [ ]:
def get_model():
    vgg = tf.keras.applications.VGG16(include_top=False, 
                                    weights=None, 
                                    input_shape=(dt.width, dt.height, 3), 
                                    pooling='avg')
    input_layer = tf.keras.Input(shape=(dt.width, dt.height, 3))
    x = vgg(input_layer)
    x = tf.keras.layers.Dense(4096, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(4096, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(input_layer, output)
    return model

In [ ]:
m = get_model()

In [ ]:
trainer.add_keras_seq_model(get_model)

In [ ]:
trainer.add_callbacks([
        tf.keras.callbacks.ReduceLROnPlateau(verbose=1)
])

In [ ]:
trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["accuracy",
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
trainer.fit(
    train,
    validation_data=val,
    epochs=30
)

In [ ]:
trainer.model.summary()

In [ ]:
train = dt.make_balaced_dataset(majority_class="chest_xray/train/PNEUMONIA",
                                minority_class="chest_xray/train/NORMAL",
                                transformations=[
                                    dt.image_flip_left_right,
                                    dt.image_central_crop,
                                    dt.image_adjust_saturation
                                ])

In [ ]:
aug_trainer = CloudTrainer(experiment_name="vgg-balanced", save_freq=30)

In [ ]:
aug_trainer.add_keras_seq_model(get_model)

In [ ]:
aug_trainer.add_callbacks([
        tf.keras.callbacks.ReduceLROnPlateau(verbose=1)
])

In [ ]:
aug_trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy",
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
aug_trainer.fit(
    train,
    validation_data=val,
    epochs=30
)

In [ ]:
aug_trainer.model.save("drive/My Drive/models/vgg-balanced/final_model.h5")